With this python-script I prepared the orignal csv-data for further usage and identified the affected artists via the Wikidata Q-Number.

In [ ]:
import re
import csv
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.error
import time
from concurrent.futures import ThreadPoolExecutor

# Clean texts
def clean_text(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[()]', '', text)
    text = re.sub(r'[\d\.]+', '', text) # Deletes numbers and points
    text = re.sub(r'/', '', text)  # Deletes slashes
    text = re.sub(r'\|.*', '', text)  # Deletes "|"
    text = re.sub(r'\'.*', '', text) # Deletes "'"

    return text

# Clean dates
def clean_date(date):
    if date is not None:
        date = re.sub(r'\[.*?\]', '', date)
        date = re.sub(r'\s+', ' ', date).strip()
        date = re.sub(r'[()]', '', date)    
        date = re.sub(r'[/\-].*', '', date) # Delete everything after a slash and the slash
        if re.match(r'^\d{4}$', date):
            return date
        if '/' in date:
            date = date.split('/')[0] + " (?)"
        if any(term in date for term in ["ca.", "um", "nach", "?"]):
            date = re.sub(r"ca\.|um|nach|\?", "", date).strip() + " (?)"
    else:
        date = "unknown"
    return date

# Function to fill empty cells with "null"
def fill_empty_cells(rows, column_names):
    for row in rows:
        for column_name in column_names:
            if column_name in row and not row[column_name].strip():
                row[column_name] = "null"
    return rows

def query_wikidata(label, lang, sparql):
    query = f"""
    SELECT ?person WHERE {{
      ?person rdfs:label "{label}"@{lang}.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],{lang}". }}
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Cache-System because of frequent time out errors
q_number_cache = {}

# SPARQL-Query to get Q-Number of artists in dataset
def get_wikidata_q_number(label):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    if label in q_number_cache:
        return q_number_cache[label]
    
    for lang in ["en", "it", "de"]:
        for _ in range(5):
            try:
                results = query_wikidata(label, lang, sparql)
                if results["results"]["bindings"]:
                    q_number = results["results"]["bindings"][0]["person"]["value"].split('/')[-1]
                    q_number_cache[label] = q_number
                    return q_number
                else:
                    query_native = f"""
                    SELECT ?person WHERE {{
                      ?person wdt:P1559 "{label}"@{lang}.
                      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],{lang}". }}
                    }}
                    LIMIT 1
                    """
                    sparql.setQuery(query_native)
                    results_native = sparql.query().convert()
                    if results_native["results"]["bindings"]:
                        q_number = results_native["results"]["bindings"][0]["person"]["value"].split('/')[-1]
                        q_number_cache[label] = q_number
                        return q_number
                    else:
                        label_temp = re.sub(r" the Younger| the Elder", "", label).strip()
                        query_temp = f"""
                        SELECT ?person WHERE {{
                          ?person rdfs:label "{label_temp}"@{lang}.
                          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],{lang}". }}
                        }}
                        LIMIT 1
                        """
                        sparql.setQuery(query_temp)
                        results_temp = sparql.query().convert()
                        if results_temp["results"]["bindings"]:
                            q_number = results_temp["results"]["bindings"][0]["person"]["value"].split('/')[-1]
                            q_number_cache[label] = q_number
                            return q_number
                        else:
                            continue
            except urllib.error.HTTPError as e:
                if e.code == 429:
                    time.sleep(3)
                    print("Musste es erneut versuchen.")
                elif e.code == 403:
                    print("HTTP Error 403: Forbidden. Überprüfen Sie die Authentifizierung und Autorisierung.")
                    return "null"
                else:
                    raise e
    
    q_number_cache[label] = "null"
    return "null"

def process_row(row):
    full_name = row.get('Hersteller/Künstler/Autor:in', '')
    date = row.get('Datierung', '')
    
    # Check if the name is not none
    if full_name and ("Unbekannt" in full_name or "Schule" in full_name or "Werkstatt" in full_name or "Meister" in full_name):
        return None
    
    row['namen_neu'] = ""
    row['Q_Nummer'] = ""
    row['datum_neu'] = ""

    # Check uncertainties of the artists and respect them in the clean data     
    name_uncertain = False
    
    if full_name is not None:
        name_uncertain = any(term in full_name for term in ["möglicherweise", "zugeschrieben", "atttributed to", "nach", "Vermutlich", "vermutlich", "im Stil von", "Stil von","Kreis des", "Kreis von", "Manner of", "Art des", "Kopie", "?"])
    if name_uncertain:
        full_name = re.sub(r"möglicherweise|zugeschrieben|attributed to|nach|Vermutlich|vermutlich|im Stil von|Stil von|Kreis des|Kreis von|Manner of|Art des|Kopie|\?", "", full_name).strip()
        full_name = re.sub(r'\s+', ' ', full_name).strip()
    else:
        name_uncertain = False
    
    date_uncertain = False

    if date is not None:
        date_uncertain = any(term in date for term in ["ca.", "um", "(?)"]) or '/' in date
    else:
        date_uncertain = False


    if full_name is not None:
        if full_name.count(',') == 1:
            family_name, given_name = full_name.split(', ')
            cleaned_family_name = clean_text(family_name)
            cleaned_given_name = clean_text(given_name)
            
            new_name = f"{cleaned_given_name} {cleaned_family_name}"
            
            # Translate or convert spelling of "the Elder" or "the Younger"
            if "der Jüngere" in new_name or "d J" in new_name:
                new_name = re.sub(r"der Jüngere|d J", "", new_name).strip() + " the Younger"
            
            if "der Ältere" in new_name or "d Ä" in new_name:
                new_name = re.sub(r"der Ältere|d Ä", "", new_name).strip() + " the Elder"
            
            new_name = re.sub(r'\s+', ' ', new_name).strip()
            
            row['namen_neu'] = new_name + " (?)" if name_uncertain else new_name
            
            q_number = get_wikidata_q_number(new_name)
            row['Q_Nummer'] = q_number

        
        elif ',' not in full_name:
            cleaned_name = clean_text(full_name)
            row['namen_neu'] = cleaned_name + " (?)" if name_uncertain else cleaned_name
            q_number = get_wikidata_q_number(cleaned_name)
            row['Q_Nummer'] = q_number
    else:
        print("full_name ist None.")
        
        row['namen_neu'] = "unknown"
        row['Q_Nummer'] = "unknown"
    
    cleaned_date = clean_date(date)
    row['datum_neu'] = cleaned_date
        
    print(f"Original: {row.get('Hersteller/Künstler/Autor:in', '')} - Bereinigt: {row['namen_neu']}")
    print(f"Originaldatum: {row.get('Datierung', '')} - Bereinigt: {row['datum_neu']}")

    return row

with open('missingArtMid.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file, delimiter=';')
    rows = list(reader)

# Delete columns, which the project does not need
columns_to_remove = ["Literatur / Quelle", "Link", "Kontakt", "Veröffentlicht seit", "Provenienz", "Inventarnummer/Signatur", "Objektart", "Datensatzart", "Meldungsart"]
for row in rows:
    for col in columns_to_remove:
        if col in row:
            del row[col]

fieldnames = list(rows[0].keys()) + ['namen_neu', 'Q_Nummer', 'datum_neu']

filtered_rows = []
with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_row, rows))

filtered_rows = [row for row in results if row is not None]

# Fill up empty columns
columns_to_fill = ['datum_neu', 'Q_Nummer', 'namen_neu']
filtered_rows = fill_empty_cells(filtered_rows, columns_to_fill)

with open('missingArtMid.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(filtered_rows)

print("Alles bereinigt.")

In [ ]:
After that I used different Python-Scripts for each type of visualization. The first one splitted the dates from the artworks in order to differenciate between certain and uncertain dates. The script also added a new column with the value "1" in order to visualize the data on a scatter plot.

In [ ]:
import pandas as pd

# Load the prepared data
df = pd.read_csv('missingArtMid.csv', delimiter=';')

# Create new column with default value "0" to differenciate the dates
df['datum_alt'] = '0'

# Function to clean the "datum_neu" column
def clean_datum_neu(value):
    if pd.isna(value):
        return '0', '0'
    elif isinstance(value, str) and ' (?)' in value:
        return '0', value
    elif isinstance(value, str) and value.isdigit() and len(value) == 4:
        return value, '0'
    else:
        return value, '0'

# Apply the function to each row in the "datum_neu" column
df['datum_neu'], df['datum_alt'] = zip(*df['datum_neu'].apply(clean_datum_neu))

# Remove " (?)" from "datum_alt" column
df['datum_alt'] = df['datum_alt'].str.replace(' (?)', '', regex=False)

# Function to check if a value contains exactly four digits
def is_four_digits(value):
    return isinstance(value, str) and value.isdigit() and len(value) == 4

# Add a new column "spalte" with default value "1"
df['spalte'] = 1

# Save the prepared data
df.to_csv('missingArt0.csv', index=False, sep=';')
print("Die Datei wurde erfolgreich bereinigt und als 'missingArt0.csv' gespeichert.")

In [ ]:
The next script adds values of latitude and longitude to the data in order to visualize them on a map. The values are gathered via a SPARQL-Query from the earlier identified Wikidata Q-Numbers of the artists and their respective properties "work location".

In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Load the prepared data
df = pd.read_csv('missingArtMid.csv', delimiter=';')

# Add new columns with default values (coordinates of Berlin as starting point of each graph)
df['lat0'] = 52.516667
df['long0'] = 13.383333

for i in range(1, 6):
    df[f'lat{i}'] = 'null'
    df[f'long{i}'] = 'null'

# Function to query Wikidata for work locations
def query_wikidata(q_number):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(f"""
    SELECT ?coordinate WHERE {{
      wd:{q_number} wdt:P937 ?location .
      ?location wdt:P625 ?coordinate .
    }}
    """)
    sparql.setReturnFormat(JSON)
    sparql.addCustomHttpHeader("User-Agent", "MyUserAgent/1.0")
    results = sparql.query().convert()
    
    coordinates = []
    for result in results["results"]["bindings"]:
        coord = result["coordinate"]["value"].strip('Point()').split()
        coordinates.append((float(coord[1]), float(coord[0])))
    
    return coordinates

# Put the coordinates from Wikidata in the data set
for index, row in df.iterrows():
    q_number = row['Q_Nummer']
    coordinates = query_wikidata(q_number)
    
    for i, (lat, long) in enumerate(coordinates[:5], start=1):
        df.at[index, f'lat{i}'] = lat
        df.at[index, f'long{i}'] = long

# Delete some other columns
df.drop(columns=['datum_neu', 'Titel'], inplace=True)

# Save the new data
df.to_csv('missingArt1.csv', index=False, sep=';')
print("Hat alles geklappt. Ich habe die Änderungen in der neuen Datei missingArt1.csv gespeichert.")

In [ ]:
Then I used this straightforward script to identify unique names in the data set and count them. With this information I can visualize the most affected artists in a lollipop chart.  

In [ ]:
import pandas as pd

# Load the prepared data
df = pd.read_csv('missingArtMid.csv', delimiter=';')

# Count the same values in one column
name_counts = df['namen_neu'].value_counts()

# Create a data set for unique names
unique_names_df = name_counts.reset_index()
unique_names_df.columns = ['namen_neu', 'anzahl_artworks']

# Implement the counted unique names in the data set 
df = df.drop_duplicates(subset=['namen_neu'])
df = df.merge(unique_names_df, on='namen_neu', how='left')

# Save the data set
df.to_csv('missingArt2.csv', index=False, sep=';')

print("Die Datei wurde erfolgreich bereinigt und als 'missingArt2.csv' gespeichert.")

In [ ]:
The following script tries to identify the art style of each painter via a SPARQL-Query. I took only into account art styles which I could identify via the movement from the artist or from one of his notable works featured in the Wikidata data set.

In [ ]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import concurrent.futures

# Load the prepared data
df = pd.read_csv('missingArtMid.csv', sep=';')

# Delete again some columns
df.drop(columns=["Lost Art ID", "Hersteller/Künstler/Autor:in", "Datierung", "Beschreibung"], inplace=True)

# Identify unique artists
counts = df['namen_neu'].value_counts()

df = df.drop_duplicates(subset=['namen_neu'])
df['anzahl_artworks'] = df['namen_neu'].map(counts)

# SPARQL-Query to obtain the property movement of the artist or one of his/her notable works
def get_genre(q_number):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    query = f"""
    SELECT ?movementLabel WHERE {{
      wd:{q_number} wdt:P135 ?movement .
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.addCustomHttpHeader("User-Agent", "MyUserAgent/1.0")
    results = sparql.query().convert()
    
    if results["results"]["bindings"]:
        return results["results"]["bindings"][0]["movementLabel"]["value"]
    
    # Only applies if no property was obtained earlier
    query = f"""
    SELECT ?notableWork WHERE {{
      wd:{q_number} wdt:P800 ?notableWork .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    if results["results"]["bindings"]:
        notable_work = results["results"]["bindings"][0]["notableWork"]["value"].split('/')[-1]
        
        query = f"""
        SELECT ?movementLabel WHERE {{
          wd:{notable_work} wdt:P135 ?movement .
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        """
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        if results["results"]["bindings"]:
            return results["results"]["bindings"][0]["movementLabel"]["value"]
    
    return None

# Function to do queries at the same time
def parallel_get_genre(q_numbers):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(get_genre, q_numbers))
    return results

df['genre'] = parallel_get_genre(df['Q_Nummer'])

# Group the data via the art style
df = df.groupby('genre').agg({
    'anzahl_artworks': 'sum',
    'namen_neu': lambda x: ', '.join(x)
}).reset_index()

# Save the data set
df.to_csv('missingArt3.csv', sep=';', index=False)

print("Die CSV-Datei wurde bereinigt und als 'missingArt3.csv' gespeichert.")

In [ ]:
For the last visualization I used the europeana search API to gather information about online repositories that provide online access to artworks from the affected painters. I had to use a JSON-Query.

In [ ]:
import requests
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
import re

# API-Key
api_key = 'PLACEHOLDER' # Subsitute with API-Key from europeana

# API-Endpoint from europeana search, in this case json und not SPARQL
search_endpoint = 'https://www.europeana.eu/api/v2/search.json'

# Load prepared data
df = pd.read_csv('missingArtMid.csv', sep=';')

# Clean the names to get better results
artists = df['namen_neu'].str.replace(r'\s*\(\?\)', '', regex=True)

# Query to count the search results for each artist and obtain the top six results
def search_artist(artist_name):
    params = {
        'query': f'proxy_dc_creator:"{artist_name}"',
        'wskey': api_key,
        'rows': 100 
    }
    response = requests.get(search_endpoint, params=params)
    if response.status_code == 200:
        results = response.json()
        domains = []
        for item in results['items']:
            is_shown_at = item.get('edmIsShownAt', [])
            if is_shown_at:
                for url in is_shown_at:
                    parsed_url = urlparse(url)
                    domain = f"{parsed_url.scheme}://{parsed_url.netloc}"
                    domains.append(domain)
        domain_counts = Counter(domains)
        top_domains = domain_counts.most_common(5)
        return top_domains
    else:
        print(f"Error: {response.status_code}")
        return []

# Apply on every artist
all_domains = []
for artist in artists:
    top_domains = search_artist(artist)
    all_domains.extend([domain for domain, count in top_domains])

overall_domain_counts = Counter(all_domains)
top_overall_domains = overall_domain_counts.most_common(6)  # Limit to the top six

for domain, count in top_overall_domains:
    print(f"Domain: {domain}, Count: {count}")

# Save the data
output_df = pd.DataFrame(top_overall_domains, columns=['domain', 'count'])
output_df.to_csv('missingArt4.csv', index=False, sep=';')

print("Die Ergebnisse wurden in 'missingArt4.csv' gespeichert.")

# Clean the earlier obtained URL for the label of the gallery
def clean_url(url):
    # Delete 'https://' oder 'http://'
    url = re.sub(r'^https?://', '', url)
    # Delete 'www.'
    url = re.sub(r'^www\.', '', url)
    # Delete Top-Level-Domain
    url = re.sub(r'\.[a-z]{2,3}$', '', url)
    return url

df_cleaned['label_domain'] = df_cleaned['domain'].apply(clean_url)

# Save the cleaned URL
df_cleaned.to_csv('bereinigte_datei.csv', index=False, sep=';')

print("Die bereinigte Datei wurde in 'bereinigte_datei.csv' gespeichert.")